In [2]:
!pip install opencv-python

In [37]:
!pip install tensorflow

In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
trainD = ImageDataGenerator(rescale=1./255)
testD = ImageDataGenerator(rescale=1./255)

In [5]:
# Preprocess all train images
train= trainD.flow_from_directory(
        'train',
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

Found 28709 images belonging to 7 classes.


In [7]:
# Preprocess all test images
test = testD.flow_from_directory(
        'test',
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

Found 7178 images belonging to 7 classes.


In [9]:
emotion_model = Sequential()

In [11]:
# Use Input layer
emotion_model.add(Input(shape=(48, 48, 1)))
emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))

In [15]:
import cv2
import numpy as np

cv2.ocl.setUseOpenCL(False)

emotion_model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [19]:
emotion_model_fit = emotion_model.fit(
    train,
    epochs=5,
    validation_data=test
)

C:\Users\GARA\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
449/449 ━━━━━━━━━━━━━━━━━━━━ 362s 793ms/step - accuracy: 0.2491 - loss: 1.8003 - val_accuracy: 0.3963 - val_loss: 1.5374
Epoch 2/5
449/449 ━━━━━━━━━━━━━━━━━━━━ 323s 720ms/step - accuracy: 0.4099 - loss: 1.5278 - val_accuracy: 0.4829 - val_loss: 1.3430
Epoch 3/5
449/449 ━━━━━━━━━━━━━━━━━━━━ 319s 710ms/step - accuracy: 0.4807 - loss: 1.3553 - val_accuracy: 0.5167 - val_loss: 1.2518
Epoch 4/5
449/449 ━━━━━━━━━━━━━━━━━━━━ 315s 700ms/step - accuracy: 0.5187 - loss: 1.2596 - val_accuracy: 0.5499 - val_loss: 1.1758
Epoch 5/5
449/449 ━━━━━━━━━━━━━━━━━━━━ 308s 686ms/step - accuracy: 0.5495 - loss: 1.1977 - val_accuracy: 0.5573 - val_loss: 1.1540


In [21]:
emotion_dict = {0: "Angry", 1: "Disgust", 2: "Fear", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprise"}

In [ ]:
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()

    if not ret:
        print("Failed to grab frame")
        break
        
    frame = cv2.resize(frame, (1280, 720))

    face_detector = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    num_faces = face_detector.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)

    
    for (x, y, w, h) in num_faces:
        cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (0, 255, 0), 4)
        roi_gray_frame = gray_frame[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48, 48)), -1), 0)

        emotion_prediction = emotion_model.predict(cropped_img)
        maxindex = int(np.argmax(emotion_prediction))
        cv2.putText(frame, emotion_dict[maxindex], (x+5, y-20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

    
    cv2.imshow('Emotion Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()